Сделайте mapper и reducer, чтобы посчитать среднее и дисперсию оценок за фильм.

Реализация через цикл (предпогаем, что мы не знаем сколько у нас фильмов):

In [1]:
import opendatasets as od
import pandas as pd
import time
from pathlib import Path
import json
from functools import reduce
from joblib import Parallel, delayed, cpu_count

In [2]:

n, mean, M2 = 0, 0.0, 0
for path in Path('imdb-user-reviews').glob('**/*'):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
        score = float(info['movieIMDbRating'])
        n += 1
        delta = score - mean
        mean += delta / n
        M2 += delta * (score - mean)

print(mean, (M2 / n) ** (1/2))

8.03 1.0517128885774865


На основе этого кода соберите mapper и reducer:

In [42]:
def mapper(path):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            loaded = json.load(f)
            score = (float(loaded['movieIMDbRating']), )
        return score

def reducer(score_data1, score_data2):
    if score_data1 is not None and score_data2 is not None:
        scores = []
        if len(score_data1) == 1:
            n, mean, M2 = 0, 0.0, 0.0
            scores.append(score_data1[0])
        else:
            n, mean, M2 = score_data1
            scores.append(score_data2[0])
        for score in scores:
            n += 1
            delta = score - mean
            mean += delta / n
            M2 += delta * (score - mean)
        return n, mean, M2
    
    
    elif score_data1 is None:
        return score_data2
    elif score_data2 is None:
        return score_data1
    else:
        return None


In [43]:
%%time
n, mean, M2 = reduce(reducer, map(mapper, Path('imdb-user-reviews').glob('**/*')))
print(mean, (M2 / n) ** (1/2))

7.944444444444445 1.0750825406152789
Wall time: 7.01 ms


In [44]:
for N in range(1, cpu_count()+1):
    start_time = time.time()
    n, mean, M2 = reduce(reducer, Parallel(n_jobs=4)(delayed(mapper)(path)for path in list(Path('').glob('**/*')))) 
    print(mean, (M2/n)**(1/2))
    print ('Количество ядер: ', N )
    print(f'Время выполнения: {round(time.time() - start_time, 2)} секунд.\n')

7.944444444444445 1.0750825406152789
Количество ядер:  1
Время выполнения: 0.07 секунд.

7.944444444444445 1.0750825406152789
Количество ядер:  2
Время выполнения: 0.06 секунд.

7.944444444444445 1.0750825406152789
Количество ядер:  3
Время выполнения: 0.04 секунд.

7.944444444444445 1.0750825406152789
Количество ядер:  4
Время выполнения: 0.04 секунд.

7.944444444444445 1.0750825406152789
Количество ядер:  5
Время выполнения: 0.04 секунд.

7.944444444444445 1.0750825406152789
Количество ядер:  6
Время выполнения: 0.04 секунд.

7.944444444444445 1.0750825406152789
Количество ядер:  7
Время выполнения: 0.04 секунд.

7.944444444444445 1.0750825406152789
Количество ядер:  8
Время выполнения: 0.04 секунд.



In [54]:
from tqdm import tqdm
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from itertools import starmap

In [55]:
POOL_THREADS = 2
def predict_function_multy_tqdm_light(vector, POOL_THREADS):
    pairs = [(data1, data2) for data1, data2 in zip(vector[:-1], vector[1:])]  # Создаём пары
    with ThreadPoolExecutor(max_workers=POOL_THREADS) as executor:
        found = list(tqdm(executor.map(lambda args: reducer(*args), pairs), total=len(pairs)))
    return found

In [56]:
%%timeit
if __name__ == '__main__':
    vector = list(Path('imdb-user-reviews').glob('**/*.json'))  
    result = predict_function_multy_tqdm_light(vector, POOL_THREADS)
    print(result)

  0%|          | 0/9 [00:00<?, ?it/s]


TypeError: object of type 'WindowsPath' has no len()